# Spark Wrangle Exercises

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

This exercises uses the `case.csv`, `dept.csv` and `source.csv` files from the san antonio 311 call dataset.

## 1.

Read the `case`, `department`, and `source` data into their own spark dataframes.

In [61]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [62]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [63]:
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

In [64]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [65]:
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

In [66]:
case.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

## 2.

Let's see how writing to the local disk works in spark:
- Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`

In [23]:
source.write.json("sources_json", mode="overwrite")

In [24]:
source.write.csv('sources_csv', mode='overwrite', header=True)

- Inspect your folder structure. What do you notice?
    - One file has no contents and is just named _SUCCESS
    - The other file has the contents of the dataframe

## 3.

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

### Case

In [67]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



- Need to convert dates to datetime format
- Yes and no columns need to be converted to booleans (`case_closed`, `case_late`)
- `council_district` needs to be converted to string

In [68]:
# rename column to make naming consistent and more readable
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [69]:
# Change date data types
print("--- Before handling dates")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

print("--- After")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-09-26 00:42:00|
|2018-01-01 00:46:00|2018-01-03 08:11:00|2018-01-05 08:30:00|
|2018-01-01 00:48:00|2018-01-02 07:57:00|2018-01-05 08:30:00|
|2018-01-01 01:29:00|2018-01-02 08:13:00|2018-01-17 08:30:00|
|2018-01-01 01:34:00|2018-01-01 13:29:00|2018-01-01 04:

In [70]:
# check yes and no columns to ensure there are only two categories
case.groupBy('case_late').count().show()

+---------+------+
|case_late| count|
+---------+------+
|      YES| 94503|
|       NO|747201|
+---------+------+



In [71]:
case.groupBy('case_closed').count().show()

+-----------+------+
|case_closed| count|
+-----------+------+
|        YES|823594|
|         NO| 18110|
+-----------+------+



In [72]:
case.groupBy('case_status').count().show()

+-----------+------+
|case_status| count|
+-----------+------+
|       Open| 18110|
|     Closed|823594|
+-----------+------+



In [73]:
# drop this column since it is same info as case_closed
case = case.drop('case_status')

In [79]:
# convert yes and no columns to booleans
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

case.select("case_closed", "case_late").show(5)

+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [81]:
# take a look at different values for council_district
case.groupBy('council_district').count().show()

+----------------+------+
|council_district| count|
+----------------+------+
|               1|119309|
|               6| 74095|
|               3|102706|
|               5|114609|
|               9| 40916|
|               4| 93778|
|               8| 42345|
|               7| 72445|
|              10| 62926|
|               2|114745|
|               0|  3830|
+----------------+------+



In [82]:
# convert council district to a string
case = case.withColumn("council_district", col("council_district").cast("string"))

In [83]:
case.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id              | 1014127333                           
 case_op

In [84]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)



### Source

In [85]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [86]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



No data type modifications needed

### Department

In [87]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [88]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



Convert last column to boolean

In [90]:
dept.groupBy('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|                YES|   31|
|                 NO|    8|
+-------------------+-----+



In [93]:
dept = dept.withColumn("dept_subject_to_SLA", expr('dept_subject_to_SLA == "YES"'))

dept.select("dept_subject_to_SLA").show(5)

+-------------------+
|dept_subject_to_SLA|
+-------------------+
|               true|
|               true|
|               true|
|               true|
|               true|
+-------------------+
only showing top 5 rows



In [94]:
dept.groupBy('dept_subject_to_SLA').count().show()

+-------------------+-----+
|dept_subject_to_SLA|count|
+-------------------+-----+
|               true|   31|
|              false|    8|
+-------------------+-----+



---

## 1.

- How old is the latest (in terms of days past SLA) currently open issue? 

In [104]:
case.where(case.case_closed == False).sort(case.num_days_late.desc()).show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id              | 1013225646                             
 case_opened_date     | 2017-01-01 13:48:00                    
 case_closed_date     | null                                   
 case_due_date        | 2017-01-17 08:30:00                    
 case_late            | true                                   
 num_days_late        | 348.6458333                            
 case_closed          | false                                  
 dept_division        | Code Enforcement                       
 service_request_type | No Address Posted                      
 SLA_days             | 15.77859954                            
 source_id            | svcCRMSS                               
 request_address      | 7299  SHADOW RIDGE, San Antonio, 78250 
 council_district     | 6                                      
only showing top 1 row



In [117]:
case.where(case.case_closed == False).select(max(col('num_days_late'))).show()

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



- How long has the oldest (in terms of days since opened) currently opened issue been open?

In [119]:
case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

case.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

case.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 12:29:00|    1407|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-03 08:11:00|    1407|             2|            2|
|       true|2018-01-01 00:48:00|2018-01-02 07:57:00|    1407|             1|            1|
|       true|2018-01-01 01:29:00|2018-01-02 08:13:00|    1407|             1|            1|
|       true|2018-01-01 01:34:00|2018-01-01 13:29:00|    1407|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+-------------------+----------------+--------+--------------+-------------+
|case_closed|   case_opened_date|case_closed_date|case_age

In [120]:
case.where(case.case_closed == False).sort(case.case_lifetime.desc()).show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 case_id              | 1013225646                             
 case_opened_date     | 2017-01-01 13:48:00                    
 case_closed_date     | null                                   
 case_due_date        | 2017-01-17 08:30:00                    
 case_late            | true                                   
 num_days_late        | 348.6458333                            
 case_closed          | false                                  
 dept_division        | Code Enforcement                       
 service_request_type | No Address Posted                      
 SLA_days             | 15.77859954                            
 source_id            | svcCRMSS                               
 request_address      | 7299  SHADOW RIDGE, San Antonio, 78250 
 council_district     | 6                                      
 case_age             | 1772                                   
 days_to_closed       | null            

## 2.

How many Stray Animal cases are there?

In [130]:
case.where(expr('service_request_type == "Stray Animal"')).count()

26760

In [124]:
case.where(col('service_request_type') == 'Stray Animal').count()

26760

In [125]:
case.where(col('case_closed') == True).where(col('service_request_type') == 'Stray Animal').count()

26745

In [126]:
case.where(col('case_closed') == False).where(col('service_request_type') == 'Stray Animal').count()

15

## 3.

How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

## 4.

Convert the `council_district` column to a string column.

## 5.

Extract the year from the `case_closed_date` column.

## 6.

Convert `num_days_late` from days to hours in new columns `num_hours_late`.

## 7.

Join the case data with the source and department data.

## 8.

Are there any cases that do not have a request source?

## 9.

What are the top 10 service request types in terms of number of requests?

## 10.

What are the top 10 service request types in terms of average days late?

## 11.

Does number of days late depend on department?

## 12.

How do number of days late depend on department and request type?

---

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.